# Final Project: MeetUp DataFrames.

In [1]:
reset -fs

In [14]:
import json
from pyspark.sql.functions import explode
from pyspark.sql.functions import col
from datetime import datetime

In [2]:
df = spark.read.json("s3a://dsci6007-firehose-final-project/meet_up2017/02/13/22/dsci6007_firehose_meet_up-1-2017-02-13-22-39-18-15eb685e-17b3-4a6a-aefb-c7309eed423c")

In [17]:
#df = spark.read.json("s3a://dsci6007-firehose-final-project/meet_up2017/02/*/*/*")

In [3]:
results_df_1 = df.select('results')

In [4]:
df.select('results.id', 'results.city').show()

+--------------------+--------------------+
|                  id|                city|
+--------------------+--------------------+
|[18590249, 185903...|[Mountain View, S...|
+--------------------+--------------------+



In [6]:
results_df_1.schema

StructType(List(StructField(results,ArrayType(StructType(List(StructField(category,StructType(List(StructField(id,LongType,true),StructField(name,StringType,true),StructField(shortname,StringType,true))),true),StructField(city,StringType,true),StructField(country,StringType,true),StructField(created,LongType,true),StructField(description,StringType,true),StructField(group_photo,StructType(List(StructField(base_url,StringType,true),StructField(highres_link,StringType,true),StructField(photo_id,LongType,true),StructField(photo_link,StringType,true),StructField(thumb_link,StringType,true),StructField(type,StringType,true))),true),StructField(id,LongType,true),StructField(join_mode,StringType,true),StructField(lat,DoubleType,true),StructField(link,StringType,true),StructField(lon,DoubleType,true),StructField(members,LongType,true),StructField(name,StringType,true),StructField(organizer,StructType(List(StructField(member_id,LongType,true),StructField(name,StringType,true),StructField(photo,

`category, city, country, created, description, group_photo, id, join_mode, lat, link, lon, members, name, organizer, rating, state, timezone, topics, urlname, utc_offset, visibility, who;'`

In [8]:
#results_df.select(results_df['results'][0]).show()

In [7]:
results_df_1.select(results_df_1['results']).show()

+--------------------+
|             results|
+--------------------+
|[[[6,education/le...|
+--------------------+



In [4]:
results_df_2 = results_df_1.select(results_df_1['results'])

## Trying to get meeting data row wise.

In [5]:
results_df_1.select(results_df_1['results']['id'][0].alias('meetup_id'), results_df_1['results']['city'][0].alias('meetup_city'),
                    results_df_1['results']['lon'][0].alias('meetup_lon'), results_df_1['results']['lat'][0].alias('meetup_lat'),
                   results_df_1['results']['name'][0].alias('meetup_name'), results_df_1['results']['members'][0].alias('meetup_members')).show()

+---------+-------------+-------------------+-----------------+--------------------+--------------+
|meetup_id|  meetup_city|         meetup_lon|       meetup_lat|         meetup_name|meetup_members|
+---------+-------------+-------------------+-----------------+--------------------+--------------+
| 18590249|Mountain View|-122.05000305175781|37.40999984741211|Bay Area PowerPoi...|            46|
+---------+-------------+-------------------+-----------------+--------------------+--------------+



In [6]:
results_df_1.select(results_df_1['results']['id'][0].alias('meetup_id'), results_df_1['results']['city'][0].alias('meetup_city'),
                   results_df_1['results']['name'][0].alias('meetup_name'), results_df_1['results']['members'][0].alias('meetup_members'),
                   results_df_1['results']['organizer'][0]['member_id'].alias('member_id'),
                   results_df_1['results']['category'][0]['id'].alias('category_id')).show()

+---------+-------------+--------------------+--------------+---------+-----------+
|meetup_id|  meetup_city|         meetup_name|meetup_members|member_id|category_id|
+---------+-------------+--------------------+--------------+---------+-----------+
| 18590249|Mountain View|Bay Area PowerPoi...|            46|144430712|          6|
+---------+-------------+--------------------+--------------+---------+-----------+



In [39]:
results_df_1.select(results_df_1['results']['id'][1].alias('meetup_id'), results_df_1['results']['city'][1].alias('meetup_city'),
                   results_df_1['results']['name'][1].alias('meetup_name'), results_df_1['results']['members'][1].alias('meetup_members'),
                   results_df_1['results']['organizer'][1]['member_id'].alias('member_id'),
                   results_df_1['results']['category'][1]['id'].alias('category_id')).show()

+---------+-----------+--------------------+--------------+---------+-----------+
|meetup_id|meetup_city|         meetup_name|meetup_members|member_id|category_id|
+---------+-----------+--------------------+--------------+---------+-----------+
| 18590368|   San Jose|Silicon Valley Fa...|           287|191340365|          8|
+---------+-----------+--------------------+--------------+---------+-----------+



In [40]:
results_df_1.withColumn('id', results_df_1['results']['id']).show()

+--------------------+--------------------+
|             results|                  id|
+--------------------+--------------------+
|[[[6,education/le...|[18590249, 185903...|
+--------------------+--------------------+



In [55]:
results_df_2.select(results_df_2['results']['id'].alias("meeting_id"), results_df_2['results']['city'].alias("city")).show()

+--------------------+--------------------+
|          meeting_id|                city|
+--------------------+--------------------+
|[18590249, 185903...|[Mountain View, S...|
+--------------------+--------------------+



In [56]:
results_df_2.select(results_df_2['results']['id'].alias("meeting_id"), results_df_2['results']['organizer']['member_id'].alias("member_id")).show()

+--------------------+--------------------+
|          meeting_id|           member_id|
+--------------------+--------------------+
|[18590249, 185903...|[144430712, 19134...|
+--------------------+--------------------+



In [57]:
results_df_2.select(results_df_2['results']['category']['id'].alias("category_id"), explode(results_df_2['results']['topics']).alias('topics')).show()

+--------------------+--------------------+
|         category_id|              topics|
+--------------------+--------------------+
|[6, 8, 31, 22, 34...|[[1001,Comedy,com...|
|[6, 8, 31, 22, 34...|[[918,Fashionista...|
|[6, 8, 31, 22, 34...|[[611,Wine,wine],...|
|[6, 8, 31, 22, 34...|[[3336,Self-Impro...|
|[6, 8, 31, 22, 34...|[[563,Open Source...|
|[6, 8, 31, 22, 34...|[[85,Science,scie...|
|[6, 8, 31, 22, 34...|[[129420,telugu l...|
|[6, 8, 31, 22, 34...|[[9131,Indian Mus...|
|[6, 8, 31, 22, 34...|[[1307,Investing,...|
|[6, 8, 31, 22, 34...|[[1244,Expat Indi...|
|[6, 8, 31, 22, 34...|[[226,Acting,acti...|
|[6, 8, 31, 22, 34...|[[15472,Network S...|
|[6, 8, 31, 22, 34...|[[2767,Book Club,...|
|[6, 8, 31, 22, 34...|[[998,Jigsaw Puzz...|
|[6, 8, 31, 22, 34...|[[1142,Current Ev...|
|[6, 8, 31, 22, 34...|[[190,Apache,apac...|
|[6, 8, 31, 22, 34...|[[885,Cashflow,ca...|
|[6, 8, 31, 22, 34...|[[10296,Women in ...|
|[6, 8, 31, 22, 34...|[[837,Expat Pakis...|
|[6, 8, 31, 22, 34...|[[10009,Wh

In [10]:
results_df_2.select(explode(results_df_2['results']['id']).alias("meeting_id")).show()

+----------+
|meeting_id|
+----------+
|  18590249|
|  18590368|
|  18591434|
|  18592202|
|  18595535|
|  18598085|
|  18606378|
|  18606597|
|  18608064|
|  18610196|
|  18610714|
|  18612333|
|  18612956|
|  18613756|
|  18616150|
|  18617234|
|  18617833|
|  18618051|
|  18618871|
|  18627258|
+----------+
only showing top 20 rows



In [11]:
results_df_2.select(explode(results_df_2['results']['city']).alias("city")).show()

+-------------+
|         city|
+-------------+
|Mountain View|
|     San Jose|
|    Sunnyvale|
|     San Jose|
|Mountain View|
|    Cupertino|
|Mountain View|
|Mountain View|
|  Santa Clara|
|      Fremont|
|  Santa Clara|
|     San Jose|
|     San Jose|
|  Santa Clara|
|Mountain View|
|Mountain View|
|     San Jose|
|Mountain View|
|  Santa Clara|
|Mountain View|
+-------------+
only showing top 20 rows



In [12]:
df_id = results_df_2.select(explode(results_df_2['results']['id']).alias("meeting_id"))

In [13]:
df_city = results_df_2.select(explode(results_df_2['results']['city']).alias("city"))

In [44]:
results_df_2.select(explode(results_df_2['results']['name']).alias("meeting_name")).show()

+--------------------+
|        meeting_name|
+--------------------+
|Bay Area PowerPoi...|
|Silicon Valley Fa...|
|       חברותא בוואלי|
|San Jose Family &...|
|Silicon Valley Da...|
|Cupertino Cleante...|
|Mountain View Eng...|
|Mountain View Car...|
|Better Investing ...|
|Bay Area Indian C...|
|Bay Area Black Th...|
|Bay Area IBM Secu...|
|Book and Coffee -...|
|    South Bay Cubers|
|India - current a...|
|Bay Area Apache F...|
|Bay Area Turn-key...|
|BOLD (Bay area Or...|
|Silicon Valley & ...|
|        IoT Security|
+--------------------+
only showing top 20 rows



In [45]:
results_df_2.select(results_df_2['results']['name'].alias('meeting_name_list'),explode(results_df_2['results']['id']).alias("meeting_id")).show()

+--------------------+----------+
|   meeting_name_list|meeting_id|
+--------------------+----------+
|[Bay Area PowerPo...|  18590249|
|[Bay Area PowerPo...|  18590368|
|[Bay Area PowerPo...|  18591434|
|[Bay Area PowerPo...|  18592202|
|[Bay Area PowerPo...|  18595535|
|[Bay Area PowerPo...|  18598085|
|[Bay Area PowerPo...|  18606378|
|[Bay Area PowerPo...|  18606597|
|[Bay Area PowerPo...|  18608064|
|[Bay Area PowerPo...|  18610196|
|[Bay Area PowerPo...|  18610714|
|[Bay Area PowerPo...|  18612333|
|[Bay Area PowerPo...|  18612956|
|[Bay Area PowerPo...|  18613756|
|[Bay Area PowerPo...|  18616150|
|[Bay Area PowerPo...|  18617234|
|[Bay Area PowerPo...|  18617833|
|[Bay Area PowerPo...|  18618051|
|[Bay Area PowerPo...|  18618871|
|[Bay Area PowerPo...|  18627258|
+--------------------+----------+
only showing top 20 rows



In [46]:
xx=results_df_2.select(results_df_2['results']['name'].alias('meeting_name_list'),explode(results_df_2['results']['id']).alias("meeting_id"))

In [47]:
xx.show()

+--------------------+----------+
|   meeting_name_list|meeting_id|
+--------------------+----------+
|[Bay Area PowerPo...|  18590249|
|[Bay Area PowerPo...|  18590368|
|[Bay Area PowerPo...|  18591434|
|[Bay Area PowerPo...|  18592202|
|[Bay Area PowerPo...|  18595535|
|[Bay Area PowerPo...|  18598085|
|[Bay Area PowerPo...|  18606378|
|[Bay Area PowerPo...|  18606597|
|[Bay Area PowerPo...|  18608064|
|[Bay Area PowerPo...|  18610196|
|[Bay Area PowerPo...|  18610714|
|[Bay Area PowerPo...|  18612333|
|[Bay Area PowerPo...|  18612956|
|[Bay Area PowerPo...|  18613756|
|[Bay Area PowerPo...|  18616150|
|[Bay Area PowerPo...|  18617234|
|[Bay Area PowerPo...|  18617833|
|[Bay Area PowerPo...|  18618051|
|[Bay Area PowerPo...|  18618871|
|[Bay Area PowerPo...|  18627258|
+--------------------+----------+
only showing top 20 rows



In [58]:
yy = results_df_2.select(results_df_2['results']['id'].alias("meeting_id"), results_df_2['results']['organizer']['member_id'].alias("member_id"))

In [59]:
yy.show()

+--------------------+--------------------+
|          meeting_id|           member_id|
+--------------------+--------------------+
|[18590249, 185903...|[144430712, 19134...|
+--------------------+--------------------+



In [65]:
yy.select(yy['meeting_id'], explode(yy['member_id']).alias('member_id')).show()

+--------------------+---------+
|          meeting_id|member_id|
+--------------------+---------+
|[18590249, 185903...|144430712|
|[18590249, 185903...|191340365|
|[18590249, 185903...|  9789749|
|[18590249, 185903...| 12785448|
|[18590249, 185903...|132425752|
|[18590249, 185903...|186352781|
|[18590249, 185903...|116454702|
|[18590249, 185903...|116454702|
|[18590249, 185903...|  9003209|
|[18590249, 185903...|  8931659|
|[18590249, 185903...|187755094|
|[18590249, 185903...|     null|
|[18590249, 185903...| 11958769|
|[18590249, 185903...| 14277841|
|[18590249, 185903...| 73983192|
|[18590249, 185903...| 16992611|
|[18590249, 185903...| 43891272|
|[18590249, 185903...| 23693311|
|[18590249, 185903...| 12636179|
|[18590249, 185903...|201071432|
+--------------------+---------+
only showing top 20 rows



In [66]:
zz=yy.select(yy['meeting_id'], explode(yy['member_id']).alias('member_id'))

In [68]:
zz.select(zz['member_id'], explode(yy['meeting_id'])).show()

+---------+--------+
|member_id|     col|
+---------+--------+
|144430712|18590249|
|144430712|18590368|
|144430712|18591434|
|144430712|18592202|
|144430712|18595535|
|144430712|18598085|
|144430712|18606378|
|144430712|18606597|
|144430712|18608064|
|144430712|18610196|
|144430712|18610714|
|144430712|18612333|
|144430712|18612956|
|144430712|18613756|
|144430712|18616150|
|144430712|18617234|
|144430712|18617833|
|144430712|18618051|
|144430712|18618871|
|144430712|18627258|
+---------+--------+
only showing top 20 rows



https://aws.amazon.com/premiumsupport/knowledge-center/rds-import-data/

http://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/PostgreSQL.Procedural.Importing.html